# Infer-17-Decision-Networks : Reseaux de Decision

**Serie** : Programmation Probabiliste avec Infer.NET (17/20)  
**Duree estimee** : 55 minutes  
**Prerequis** : Notebooks 3 (Factor Graphs), 14-16 (Utilite)

---

## Objectifs

- Etendre les reseaux bayesiens avec **noeuds de decision et d'utilite**
- Calculer la **politique optimale** dans un reseau de decision
- Comprendre les **arcs informationnels**
- Modeliser des **decisions sequentielles**

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-16-Decision-Multi-Attribute](Infer-16-Decision-Multi-Attribute.ipynb) | [Infer-18-Decision-Value-Information](Infer-18-Decision-Value-Information.ipynb) |

---

## 1. Des Reseaux Bayesiens aux Reseaux de Decision

### Rappel : Reseaux Bayesiens

Un reseau bayesien represente des **relations probabilistes** entre variables :

```
  [Maladie] ----> [Symptome]
       \--------> [Test]
```

### Extension : Reseaux de Decision (Influence Diagrams)

Un reseau de decision ajoute :

- **Noeuds de decision** : choix de l'agent
- **Noeuds d'utilite** : consequences des choix
- **Arcs informationnels** : ce que l'agent sait au moment de decider

```
  [Maladie] ----> [Test Result]
       |              |
       v              v
  <Traitement> ----> ((Utilite))
```

In [1]:
// Installation Infer.NET
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;

Console.WriteLine("Infer.NET charge !");

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET charge !


## 2. Types de Noeuds

### Convention graphique

| Type | Forme | Symbole | Description |
|------|-------|---------|-------------|
| **Chance** | Ovale/Cercle | [X] | Variable aleatoire |
| **Decision** | Rectangle | <D> | Choix controle par l'agent |
| **Utilite** | Losange/Hexagone | ((U)) | Fonction de valeur |

### Arcs

| Type d'arc | Signification |
|------------|---------------|
| Chance → Chance | Dependance probabiliste |
| Chance → Decision | Arc informationnel (l'agent observe) |
| Chance → Utilite | La variable affecte l'utilite |
| Decision → Chance | La decision influence le resultat |
| Decision → Utilite | La decision a un cout/benefice direct |

In [2]:
// Representation d'un reseau de decision simple

public enum NodeType { Chance, Decision, Utility }

public class DecisionNode
{
    public string Name { get; set; }
    public NodeType Type { get; set; }
    public List<string> Parents { get; set; } = new List<string>();
    
    public string ToSymbol()
    {
        return Type switch
        {
            NodeType.Chance => $"[{Name}]",
            NodeType.Decision => $"<{Name}>",
            NodeType.Utility => $"(({Name}))",
            _ => Name
        };
    }
}

// Exemple : Decision medicale
var network = new List<DecisionNode>
{
    new DecisionNode { Name = "Maladie", Type = NodeType.Chance },
    new DecisionNode { Name = "Test", Type = NodeType.Chance, Parents = { "Maladie" } },
    new DecisionNode { Name = "Traiter", Type = NodeType.Decision, Parents = { "Test" } },
    new DecisionNode { Name = "Sante", Type = NodeType.Chance, Parents = { "Maladie", "Traiter" } },
    new DecisionNode { Name = "Utilite", Type = NodeType.Utility, Parents = { "Sante", "Traiter" } },
};

Console.WriteLine("Reseau de decision : Diagnostic Medical\n");
foreach (var node in network)
{
    string parents = node.Parents.Any() ? $" <- {string.Join(", ", node.Parents)}" : "";
    Console.WriteLine($"  {node.ToSymbol()}{parents}");
}

Reseau de decision : Diagnostic Medical

  [Maladie]
  [Test] <- Maladie
  <Traiter> <- Test
  [Sante] <- Maladie, Traiter
  ((Utilite)) <- Sante, Traiter


## 3. Arcs Informationnels

### Concept cle

Un arc informationnel vers un noeud de decision indique **ce que l'agent sait** au moment de prendre cette decision.

### Exemple

```
[Resultat Test] ----> <Traitement>
```

Signifie : L'agent connait le resultat du test **avant** de decider du traitement.

### No-forgetting assumption

Dans les decisions sequentielles, on suppose generalement que l'agent **n'oublie pas** :
- Toutes les observations passees
- Toutes les decisions passees

In [3]:
// Impact des arcs informationnels

Console.WriteLine("Scenarios d'information :\n");

// Prior sur la maladie
double pMaladie = 0.1;
double sensibilite = 0.95; // P(test+|malade)
double specificite = 0.90; // P(test-|sain)

// Utilites
double U_traiter_malade = 90;    // Guerison
double U_traiter_sain = 70;      // Effets secondaires inutiles
double U_pasTraiter_malade = 10; // Maladie non traitee
double U_pasTraiter_sain = 100;  // Parfait

// Scenario 1 : Pas d'information (decision a priori)
double EU_traiter_apriori = pMaladie * U_traiter_malade + (1 - pMaladie) * U_traiter_sain;
double EU_pasTraiter_apriori = pMaladie * U_pasTraiter_malade + (1 - pMaladie) * U_pasTraiter_sain;

Console.WriteLine("Scenario 1 : Decision SANS information (pas de test)");
Console.WriteLine($"  P(malade) = {pMaladie:P0}");
Console.WriteLine($"  E[U(traiter)] = {EU_traiter_apriori:F1}");
Console.WriteLine($"  E[U(pas traiter)] = {EU_pasTraiter_apriori:F1}");
Console.WriteLine($"  => Decision : {(EU_traiter_apriori > EU_pasTraiter_apriori ? "TRAITER" : "PAS TRAITER")}\n");

// Scenario 2 : Decision AVEC information (apres test)
// Calculer P(malade|test+) et P(malade|test-) par Bayes
double pTestPositif = pMaladie * sensibilite + (1 - pMaladie) * (1 - specificite);
double pMaladeSachantTestPositif = (pMaladie * sensibilite) / pTestPositif;
double pMaladeSachantTestNegatif = (pMaladie * (1 - sensibilite)) / (1 - pTestPositif);

Console.WriteLine("Scenario 2 : Decision AVEC information (apres test)");
Console.WriteLine($"  P(test+) = {pTestPositif:P1}");
Console.WriteLine($"  P(malade|test+) = {pMaladeSachantTestPositif:P1}");
Console.WriteLine($"  P(malade|test-) = {pMaladeSachantTestNegatif:P1}");

// Decision si test positif
double EU_traiter_testPos = pMaladeSachantTestPositif * U_traiter_malade + (1 - pMaladeSachantTestPositif) * U_traiter_sain;
double EU_pasTraiter_testPos = pMaladeSachantTestPositif * U_pasTraiter_malade + (1 - pMaladeSachantTestPositif) * U_pasTraiter_sain;

Console.WriteLine($"  Si test+ : traiter={EU_traiter_testPos:F1}, pas traiter={EU_pasTraiter_testPos:F1} => {(EU_traiter_testPos > EU_pasTraiter_testPos ? "TRAITER" : "PAS TRAITER")}");

// Decision si test negatif
double EU_traiter_testNeg = pMaladeSachantTestNegatif * U_traiter_malade + (1 - pMaladeSachantTestNegatif) * U_traiter_sain;
double EU_pasTraiter_testNeg = pMaladeSachantTestNegatif * U_pasTraiter_malade + (1 - pMaladeSachantTestNegatif) * U_pasTraiter_sain;

Console.WriteLine($"  Si test- : traiter={EU_traiter_testNeg:F1}, pas traiter={EU_pasTraiter_testNeg:F1} => {(EU_traiter_testNeg > EU_pasTraiter_testNeg ? "TRAITER" : "PAS TRAITER")}");

Scenarios d'information :

Scenario 1 : Decision SANS information (pas de test)
  P(malade) = 10 %
  E[U(traiter)] = 72,0
  E[U(pas traiter)] = 91,0
  => Decision : PAS TRAITER

Scenario 2 : Decision AVEC information (apres test)
  P(test+) = 18,5 %
  P(malade|test+) = 51,4 %
  P(malade|test-) = 0,6 %
  Si test+ : traiter=80,3, pas traiter=53,8 => TRAITER
  Si test- : traiter=70,1, pas traiter=99,4 => PAS TRAITER


## 4. Calcul de la Politique Optimale

### Algorithme general

1. **Ordonner** les noeuds de decision (ordre temporel)
2. Pour chaque noeud de decision, du dernier au premier :
   - Enumerer les valeurs possibles des parents informationnels
   - Pour chaque configuration, calculer E[U] pour chaque choix
   - Selectionner le choix qui maximise E[U]
3. La **politique** est une fonction : observations → decision

In [4]:
// Implementation d'un solveur de reseau de decision simple

public class SimpleDecisionNetwork
{
    // Noeud chance : Maladie (M)
    public double P_M { get; set; } = 0.1; // P(malade)
    
    // Noeud chance : Test (T) | M
    public double P_TposGivenM { get; set; } = 0.95;  // Sensibilite
    public double P_TposGivenNotM { get; set; } = 0.10; // 1-Specificite
    
    // Noeud decision : Traiter (D) avec parent informationnel T
    // Options : D=true (traiter), D=false (pas traiter)
    
    // Noeud utilite : U(M, D)
    public double U_traiter_malade { get; set; } = 90;
    public double U_traiter_sain { get; set; } = 70;
    public double U_pasTraiter_malade { get; set; } = 10;
    public double U_pasTraiter_sain { get; set; } = 100;
    
    public double GetUtility(bool malade, bool traiter)
    {
        if (traiter && malade) return U_traiter_malade;
        if (traiter && !malade) return U_traiter_sain;
        if (!traiter && malade) return U_pasTraiter_malade;
        return U_pasTraiter_sain;
    }
    
    public (Dictionary<bool, bool> policy, double expectedUtility) Solve()
    {
        // Politique : Test result -> Decision
        var policy = new Dictionary<bool, bool>();
        
        // P(T+)
        double pTpos = P_M * P_TposGivenM + (1 - P_M) * P_TposGivenNotM;
        double pTneg = 1 - pTpos;
        
        // Pour T+ : calculer decision optimale
        double pM_givenTpos = (P_M * P_TposGivenM) / pTpos;
        double EU_traiter_Tpos = pM_givenTpos * U_traiter_malade + (1 - pM_givenTpos) * U_traiter_sain;
        double EU_pasTraiter_Tpos = pM_givenTpos * U_pasTraiter_malade + (1 - pM_givenTpos) * U_pasTraiter_sain;
        policy[true] = EU_traiter_Tpos > EU_pasTraiter_Tpos;
        double bestU_Tpos = Math.Max(EU_traiter_Tpos, EU_pasTraiter_Tpos);
        
        // Pour T- : calculer decision optimale
        double pM_givenTneg = (P_M * (1 - P_TposGivenM)) / pTneg;
        double EU_traiter_Tneg = pM_givenTneg * U_traiter_malade + (1 - pM_givenTneg) * U_traiter_sain;
        double EU_pasTraiter_Tneg = pM_givenTneg * U_pasTraiter_malade + (1 - pM_givenTneg) * U_pasTraiter_sain;
        policy[false] = EU_traiter_Tneg > EU_pasTraiter_Tneg;
        double bestU_Tneg = Math.Max(EU_traiter_Tneg, EU_pasTraiter_Tneg);
        
        // Utilite esperee totale
        double totalEU = pTpos * bestU_Tpos + pTneg * bestU_Tneg;
        
        return (policy, totalEU);
    }
}

var dn = new SimpleDecisionNetwork();
var (policy, eu) = dn.Solve();

Console.WriteLine("=== Resolution du Reseau de Decision ===");
Console.WriteLine();
Console.WriteLine("Politique optimale :");
Console.WriteLine($"  Si test POSITIF : {(policy[true] ? "TRAITER" : "PAS TRAITER")}");
Console.WriteLine($"  Si test NEGATIF : {(policy[false] ? "TRAITER" : "PAS TRAITER")}");
Console.WriteLine();
Console.WriteLine($"Utilite esperee : {eu:F2}");

=== Resolution du Reseau de Decision ===

Politique optimale :
  Si test POSITIF : TRAITER
  Si test NEGATIF : PAS TRAITER

Utilite esperee : 95,90


## 5. Exemple : Investissement avec Test de Marche

### Scenario

Un investisseur considere un projet. Le marche peut etre favorable ou non.
Il peut commander une etude de marche avant de decider.

```
[Marche] -----> [Etude] -----> <Investir>
    |                              |
    +----------------------------> ((Profit))
```

In [5]:
// Reseau de decision : Investissement

// Marche (M) : Favorable (F) ou Defavorable (D)
double pFavorable = 0.4;

// Etude de marche (E) : Positive ou Negative
// Qualite de l'etude
double pEposGivenF = 0.8;  // Detecte correctement marche favorable
double pEposGivenD = 0.3;  // Faux positif

// Profits selon marche et decision
double profit_investir_favorable = 500000;    // Gros profit
double profit_investir_defavorable = -200000; // Grosse perte
double profit_pasInvestir = 0;                // Statu quo
double cout_etude = 20000;                    // Cout de l'etude

// Calculer la politique optimale
Console.WriteLine("=== Decision d'Investissement ===");
Console.WriteLine($"P(marche favorable) = {pFavorable:P0}");
Console.WriteLine($"Cout de l'etude = {cout_etude:N0} EUR\n");

// Option 1 : Pas d'etude, decision directe
double EU_investir_direct = pFavorable * profit_investir_favorable + 
                            (1 - pFavorable) * profit_investir_defavorable;
double EU_pasInvestir_direct = profit_pasInvestir;
double bestEU_sansEtude = Math.Max(EU_investir_direct, EU_pasInvestir_direct);

Console.WriteLine("Option 1 : Decision directe (sans etude)");
Console.WriteLine($"  E[Profit(investir)] = {EU_investir_direct:N0} EUR");
Console.WriteLine($"  E[Profit(pas investir)] = {EU_pasInvestir_direct:N0} EUR");
Console.WriteLine($"  => Decision : {(EU_investir_direct > EU_pasInvestir_direct ? "INVESTIR" : "NE PAS INVESTIR")}");
Console.WriteLine($"  => E[Profit] = {bestEU_sansEtude:N0} EUR\n");

// Option 2 : Avec etude
double pEpos = pFavorable * pEposGivenF + (1 - pFavorable) * pEposGivenD;
double pEneg = 1 - pEpos;

// Si etude positive
double pF_givenEpos = (pFavorable * pEposGivenF) / pEpos;
double EU_investir_Epos = pF_givenEpos * profit_investir_favorable + 
                          (1 - pF_givenEpos) * profit_investir_defavorable;
bool investir_siEpos = EU_investir_Epos > profit_pasInvestir;
double bestEU_Epos = Math.Max(EU_investir_Epos, profit_pasInvestir);

// Si etude negative
double pF_givenEneg = (pFavorable * (1 - pEposGivenF)) / pEneg;
double EU_investir_Eneg = pF_givenEneg * profit_investir_favorable + 
                          (1 - pF_givenEneg) * profit_investir_defavorable;
bool investir_siEneg = EU_investir_Eneg > profit_pasInvestir;
double bestEU_Eneg = Math.Max(EU_investir_Eneg, profit_pasInvestir);

double EU_avecEtude = pEpos * bestEU_Epos + pEneg * bestEU_Eneg - cout_etude;

Console.WriteLine("Option 2 : Avec etude de marche");
Console.WriteLine($"  P(etude positive) = {pEpos:P1}");
Console.WriteLine($"  Si E+ : P(favorable|E+) = {pF_givenEpos:P1} => {(investir_siEpos ? "INVESTIR" : "NE PAS INVESTIR")}");
Console.WriteLine($"  Si E- : P(favorable|E-) = {pF_givenEneg:P1} => {(investir_siEneg ? "INVESTIR" : "NE PAS INVESTIR")}");
Console.WriteLine($"  => E[Profit avec etude] = {EU_avecEtude:N0} EUR\n");

Console.WriteLine("=== Comparaison ===");
Console.WriteLine($"Sans etude : {bestEU_sansEtude:N0} EUR");
Console.WriteLine($"Avec etude : {EU_avecEtude:N0} EUR");
Console.WriteLine($"=> Valeur de l'etude : {EU_avecEtude - bestEU_sansEtude:N0} EUR");
Console.WriteLine($"=> Decision : {(EU_avecEtude > bestEU_sansEtude ? "COMMANDER L'ETUDE" : "DECIDER DIRECTEMENT")}");

=== Decision d'Investissement ===
P(marche favorable) = 40 %
Cout de l'etude = 20 000 EUR

Option 1 : Decision directe (sans etude)
  E[Profit(investir)] = 80 000 EUR
  E[Profit(pas investir)] = 0 EUR
  => Decision : INVESTIR
  => E[Profit] = 80 000 EUR

Option 2 : Avec etude de marche
  P(etude positive) = 50,0 %
  Si E+ : P(favorable|E+) = 64,0 % => INVESTIR
  Si E- : P(favorable|E-) = 16,0 % => NE PAS INVESTIR
  => E[Profit avec etude] = 104 000 EUR

=== Comparaison ===
Sans etude : 80 000 EUR
Avec etude : 104 000 EUR
=> Valeur de l'etude : 24 000 EUR
=> Decision : COMMANDER L'ETUDE


## 6. Decisions Sequentielles

### Ordre des decisions

Quand il y a plusieurs noeuds de decision, leur **ordre temporel** doit etre specifie.

### Exemple : Test puis Traitement

```
<Faire Test?> ----> [Resultat] ----> <Traiter?> ----> ((Utilite))
                                          ^
                                          |
[Maladie] ---------------------------------+
```

Deux decisions sequentielles :
1. D'abord : Faire le test ou non ?
2. Ensuite : Traiter ou non ? (en connaissant le resultat si test fait)

In [6]:
// Decision sequentielle : Faire test puis traiter

// Parametres
double pMaladie_seq = 0.2;
double sensibilite_seq = 0.90;
double specificite_seq = 0.85;
double cout_test = 50;

// Utilites (sans cout test)
double U_traiterMalade = 100;
double U_traiterSain = 60;
double U_pasTraiterMalade = 20;
double U_pasTraiterSain = 100;

// Resolution par backward induction
Console.WriteLine("=== Decision Sequentielle : Test puis Traitement ===\n");

// D'abord : calculer politique optimale pour D2 (traiter) sachant test fait
double pTpos_seq = pMaladie_seq * sensibilite_seq + (1 - pMaladie_seq) * (1 - specificite_seq);

double pM_Tpos = (pMaladie_seq * sensibilite_seq) / pTpos_seq;
double EU_traiter_Tpos = pM_Tpos * U_traiterMalade + (1 - pM_Tpos) * U_traiterSain;
double EU_pasTraiter_Tpos = pM_Tpos * U_pasTraiterMalade + (1 - pM_Tpos) * U_pasTraiterSain;
double bestU_Tpos = Math.Max(EU_traiter_Tpos, EU_pasTraiter_Tpos) - cout_test;

double pM_Tneg = (pMaladie_seq * (1 - sensibilite_seq)) / (1 - pTpos_seq);
double EU_traiter_Tneg = pM_Tneg * U_traiterMalade + (1 - pM_Tneg) * U_traiterSain;
double EU_pasTraiter_Tneg = pM_Tneg * U_pasTraiterMalade + (1 - pM_Tneg) * U_pasTraiterSain;
double bestU_Tneg = Math.Max(EU_traiter_Tneg, EU_pasTraiter_Tneg) - cout_test;

double EU_faireTest = pTpos_seq * bestU_Tpos + (1 - pTpos_seq) * bestU_Tneg;

// Si pas de test : decision a priori
double EU_traiter_apriori = pMaladie_seq * U_traiterMalade + (1 - pMaladie_seq) * U_traiterSain;
double EU_pasTraiter_apriori = pMaladie_seq * U_pasTraiterMalade + (1 - pMaladie_seq) * U_pasTraiterSain;
double EU_pasTest = Math.Max(EU_traiter_apriori, EU_pasTraiter_apriori);

Console.WriteLine("Etape 2 : Politique de traitement si test fait");
Console.WriteLine($"  Si T+ : P(M|T+)={pM_Tpos:P1} => {(EU_traiter_Tpos > EU_pasTraiter_Tpos ? "Traiter" : "Pas traiter")}");
Console.WriteLine($"  Si T- : P(M|T-)={pM_Tneg:P1} => {(EU_traiter_Tneg > EU_pasTraiter_Tneg ? "Traiter" : "Pas traiter")}\n");

Console.WriteLine("Etape 1 : Decision de faire le test");
Console.WriteLine($"  E[U | faire test] = {EU_faireTest:F1}");
Console.WriteLine($"  E[U | pas test] = {EU_pasTest:F1} ({(EU_traiter_apriori > EU_pasTraiter_apriori ? "traiter" : "pas traiter")})\n");

Console.WriteLine("=== Politique Optimale Globale ===");
Console.WriteLine($"D1 : {(EU_faireTest > EU_pasTest ? "FAIRE LE TEST" : "NE PAS FAIRE LE TEST")}");
if (EU_faireTest > EU_pasTest)
{
    Console.WriteLine($"D2 : Si T+ => {(EU_traiter_Tpos > EU_pasTraiter_Tpos ? "TRAITER" : "PAS TRAITER")}");
    Console.WriteLine($"     Si T- => {(EU_traiter_Tneg > EU_pasTraiter_Tneg ? "TRAITER" : "PAS TRAITER")}");
}
else
{
    Console.WriteLine($"D2 : {(EU_traiter_apriori > EU_pasTraiter_apriori ? "TRAITER" : "PAS TRAITER")} (sans test)");
}

=== Decision Sequentielle : Test puis Traitement ===

Etape 2 : Politique de traitement si test fait
  Si T+ : P(M|T+)=60,0 % => Traiter
  Si T- : P(M|T-)=2,9 % => Pas traiter

Etape 1 : Decision de faire le test
  E[U | faire test] = 43,6
  E[U | pas test] = 84,0 (pas traiter)

=== Politique Optimale Globale ===
D1 : NE PAS FAIRE LE TEST
D2 : PAS TRAITER (sans test)


## 7. Implementation avec Infer.NET

Infer.NET n'a pas de support natif pour les noeuds de decision, mais nous pouvons :

1. Modeliser les variables de chance
2. Enumerer les decisions
3. Calculer E[U] pour chaque configuration

In [ ]:
// Reseau de decision avec Infer.NET

// Probleme : Diagnostic et traitement

// Variables de chance
Variable<bool> maladie = Variable.Bernoulli(0.15).Named("maladie");

// Test (conditionnel a la maladie)
Variable<bool> testPositif = Variable.New<bool>().Named("test");
using (Variable.If(maladie))
    testPositif.SetTo(Variable.Bernoulli(0.92)); // Sensibilite
using (Variable.IfNot(maladie))
    testPositif.SetTo(Variable.Bernoulli(0.12)); // 1-Specificite

// Inference
InferenceEngine engine = new InferenceEngine();
engine.Compiler.CompilerChoice = Microsoft.ML.Probabilistic.Compiler.CompilerChoice.Roslyn;

// Enumerer les observations possibles et calculer E[U] pour chaque decision
var utilites = new Dictionary<string, double>
{
    {"traiter_malade", 85},
    {"traiter_sain", 65},
    {"pasTraiter_malade", 15},
    {"pasTraiter_sain", 100}
};

Console.WriteLine("=== Reseau de Decision avec Infer.NET ===\n");

foreach (bool obsTest in new[] { true, false })
{
    // Observer le test
    testPositif.ObservedValue = obsTest;
    
    // Inferrer P(maladie|test)
    Bernoulli posteriorMaladie = engine.Infer<Bernoulli>(maladie);
    double pM = posteriorMaladie.GetProbTrue();
    
    // Calculer E[U] pour chaque decision
    double EU_traiter = pM * utilites["traiter_malade"] + (1 - pM) * utilites["traiter_sain"];
    double EU_pasTraiter = pM * utilites["pasTraiter_malade"] + (1 - pM) * utilites["pasTraiter_sain"];
    
    string decision = EU_traiter > EU_pasTraiter ? "TRAITER" : "PAS TRAITER";
    
    Console.WriteLine($"Observation : test = {(obsTest ? "POSITIF" : "NEGATIF")}");
    Console.WriteLine($"  P(maladie|test) = {pM:P1}");
    Console.WriteLine($"  E[U(traiter)] = {EU_traiter:F1}");
    Console.WriteLine($"  E[U(pas traiter)] = {EU_pasTraiter:F1}");
    Console.WriteLine($"  => Decision optimale : {decision}\n");
    
    // Nettoyer l'observation pour la prochaine iteration
    testPositif.ClearObservedValue();
}

### 7.1 Visualisation du Factor Graph

Infer.NET peut generer le **graphe de facteurs** du modele. C'est utile pour :
- Verifier la structure du modele
- Comprendre les dependances
- Debugger les erreurs d'inference

In [ ]:
// Visualisation du graphe de facteurs du modele de diagnostic

// Recreer le modele (nouvelle instance pour generation du graphe)
Variable<bool> maladieFG = Variable.Bernoulli(0.15).Named("Maladie");
Variable<bool> testFG = Variable.New<bool>().Named("TestResult");

using (Variable.If(maladieFG))
    testFG.SetTo(Variable.Bernoulli(0.92));
using (Variable.IfNot(maladieFG))
    testFG.SetTo(Variable.Bernoulli(0.12));

// Activer la generation du graphe de facteurs
InferenceEngine engineFG = new InferenceEngine();
engineFG.Compiler.CompilerChoice = Microsoft.ML.Probabilistic.Compiler.CompilerChoice.Roslyn;
engineFG.ShowFactorGraph = true;  // Active la generation du graphe

Console.WriteLine("=== Generation du Factor Graph ===\n");
Console.WriteLine("Le graphe de facteurs sera genere lors de l'inference.");
Console.WriteLine("Sur un environnement avec Graphviz installe, un fichier .dgml sera cree.\n");

// Effectuer l'inference (declenche la generation du graphe)
try 
{
    var posterior = engineFG.Infer<Bernoulli>(maladieFG);
    Console.WriteLine($"Inference reussie : P(Maladie) = {posterior}");
    Console.WriteLine();
    Console.WriteLine("Structure du modele :");
    Console.WriteLine("  [Maladie] -----> [TestResult]");
    Console.WriteLine("      |                 |");
    Console.WriteLine("      v                 v");
    Console.WriteLine("  (Bernoulli)    (Conditionnel)");
    Console.WriteLine();
    Console.WriteLine("Dans le factor graph :");
    Console.WriteLine("  - Noeuds ronds = Variables (Maladie, TestResult)");
    Console.WriteLine("  - Noeuds carres = Facteurs (Prior, Likelihood)");
    Console.WriteLine("  - Aretes = Connexions facteur-variable");
}
catch (Exception ex)
{
    Console.WriteLine($"Note : {ex.Message}");
    Console.WriteLine("La visualisation graphique necessite Graphviz.");
}

## 8. Exercice : Reseau de Decision Personnalise

### Enonce

Modelisez le reseau de decision suivant :

Un etudiant doit decider s'il revise ou pas pour un examen.
- Variable aleatoire : Difficulte de l'examen (Facile/Difficile)
- Decision : Reviser (cout en temps) ou pas
- Utilite : Note obtenue moins cout de revision

In [8]:
// Exercice : Reseau de decision Etudiant

// Parametres
double pDifficile = 0.4; // P(examen difficile)

// Notes esperees
double note_revise_facile = 18;
double note_revise_difficile = 14;
double note_pasRevise_facile = 14;
double note_pasRevise_difficile = 8;

// Cout de revision (en points d'utilite)
double coutRevision = 2;

// Calcul des utilites esperees
// A completer...

double EU_reviser = (1 - pDifficile) * (note_revise_facile - coutRevision) +
                    pDifficile * (note_revise_difficile - coutRevision);

double EU_pasReviser = (1 - pDifficile) * note_pasRevise_facile +
                       pDifficile * note_pasRevise_difficile;

Console.WriteLine("=== Decision Etudiant ===\n");
Console.WriteLine($"P(examen difficile) = {pDifficile:P0}");
Console.WriteLine($"Cout revision = {coutRevision} points\n");

Console.WriteLine($"E[U(reviser)] = {EU_reviser:F1}");
Console.WriteLine($"E[U(pas reviser)] = {EU_pasReviser:F1}\n");

Console.WriteLine($"=> Decision optimale : {(EU_reviser > EU_pasReviser ? "REVISER" : "NE PAS REVISER")}");

// Analyse de sensibilite : a partir de quel cout la decision change ?
double coutSeuil = (1 - pDifficile) * note_revise_facile + pDifficile * note_revise_difficile - EU_pasReviser;
Console.WriteLine($"\nCout seuil (indifference) : {coutSeuil:F1} points");

=== Decision Etudiant ===

P(examen difficile) = 40 %
Cout revision = 2 points

E[U(reviser)] = 14,4
E[U(pas reviser)] = 11,6

=> Decision optimale : REVISER

Cout seuil (indifference) : 4,8 points


## 9. Resume

| Concept | Description |
|---------|-------------|
| **Reseau de decision** | Extension des reseaux bayesiens avec decisions et utilites |
| **Noeuds de decision** | Choix controles par l'agent |
| **Noeuds d'utilite** | Consequences des choix |
| **Arcs informationnels** | Ce que l'agent sait au moment de decider |
| **Politique** | Fonction observations → decisions |
| **Backward induction** | Resolution du dernier au premier noeud de decision |

---

## Pour aller plus loin

| Si vous voulez... | Consultez... |
|-------------------|-------------|
| Calculer la valeur de l'information | [Infer-18-Decision-Value-Information](Infer-18-Decision-Value-Information.ipynb) |
| Decisions robustes | [Infer-19-Decision-Expert-Systems](Infer-19-Decision-Expert-Systems.ipynb) |
| Decisions sequentielles (MDPs) | [Infer-20-Decision-Sequential](Infer-20-Decision-Sequential.ipynb) |

---

## Prochaine etape

Dans [Infer-18-Decision-Value-Information](Infer-18-Decision-Value-Information.ipynb), nous verrons :

- La valeur de l'information parfaite (EVPI)
- La valeur de l'information d'echantillon (EVSI)
- Des applications : droits de forage, chasse au tresor

---

## References

- Howard & Matheson (1984) : Influence Diagrams
- Shachter (1986) : Evaluating Influence Diagrams
- Russell & Norvig : AI, Chapter 16.5